# Постановка задачи

HR-агентство хочет провести исследование на основе данных о зарплатах в сфере Data Science за 2020–2022 годы и выяснить:

- наблюдается ли ежегодный рост зарплат у специалистов Data Scientist;
- как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году;
- как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров;
- есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании.

# Описание входных данных

**work_year** - год, в котором была выплачена зарплата.

**experience_level** - опыт работы на этой должности в течение года со следующими возможными значениями:

EN — Entry-level/Junior;

MI — Mid-level/Intermediate;

SE — Senior-level/Expert;

EX — Executive-level/Director.


**employment_type** - тип трудоустройства для этой роли:

PT — неполный рабочий день;

FT — полный рабочий день;

CT — контракт;

FL — фриланс.


**job_title** - роль, в которой соискатель работал в течение года.


**salary** - общая выплаченная валовая сумма заработной платы.


**salary_currency** - валюта выплачиваемой заработной платы в виде кода валюты ISO 4217.


**salary_in_usd**- зарплата в долларах США (валютный курс, делённый на среднее значение курса доллара США за соответствующий год через fxdata.foorilla.com).


**employee_residence** - основная страна проживания сотрудника в течение рабочего года в виде кода страны ISO 3166.


**remote_ratio** - общий объём работы, выполняемой удалённо. Возможные значения:

0 — удалённой работы нет (менее 20 %);

50 — частично удалённая работа;

100 — полностью удалённая работа (более 80 %).


**company_location** - страна главного офиса работодателя или филиала по контракту в виде кода страны ISO 3166.


**company_size** - среднее количество людей, работавших в компании в течение года:

S — менее 50 сотрудников (небольшая компания);

M — от 50 до 250 сотрудников (средняя компания);

L — более 250 сотрудников (крупная компания).

In [605]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from enum import Enum

In [606]:
# отображать .png изображения, нативные графики plotly или всё вместе
class render_type(Enum):
    plotly  = 1
    png     = 2,
    all     = 3


class render_plot():

    @staticmethod
    def render(plot, file, width, height, type = render_type.all):

        if type == render_type.png or type == render_type.all:
            pio.write_image(plot, file, scale = 1, width = width, height = height)

        if type == render_type.all or type == render_type.plotly:
            plot.show()


# для смены формата отображения графиков:
# если код графика был изменен - очистить папку images
# поменять значение типа, запустить ячейки, сохранить изменения, перезагрузить .ipynb файл
current_render_type = render_type.png


# локализация для графиков
locale = {
    'work_year':                    'Год выплаты ЗП',

    'experience_level':             'Опыт работы',
    'EN':                           'Entry-level/Junior',
    'MI':                           'Mid-level/Intermediate',
    'SE':                           'Senior-level/Expert',
    'EX':                           'Executive-level/Director',

    'employment_type':              'Тип трудоустройства',
    'PT':                           'Неполный рабочий день',
    'FT':                           'Полный рабочий день',
    'CT':                           'Контракт',
    'FL':                           'Фриланс',

    'job_title':                    'Роль',
    'salary':                       'ЗП в рублях',
    'salary_currency':              'Валюта',
    'salary_in_usd' :               'ЗП в долларах',
    'employee_residence' :          'Страна проживания',

    'remote_ratio':                 'Общий объем удалённой работы',
    0:                              'Нет',
    50:                             'Частично удалённая',
    100:                            'Полностью удалённая',

    'company_location':             'Страна главного офиса работодателя',

    'company_size':                 'Среднее количество сотрудников',
    'S':                            'Менее 50',
    'M':                            'От 50 до 250',
    'L':                            'Более 250'
}

# 1. Загрузка и обработка данных

In [607]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/denis-marchenkov-sf/assets/master/ds_salaries.csv',
    index_col = 'Unnamed: 0'
    )

df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


### Общая информация о таблице

In [608]:
print('\nОбщая информация о таблице:\n')
df.info();
print(f'\nРазмерность таблицы: {df.shape}\n')

print('\nСтатистические характеристики признаков:\n')
df.describe(include = 'all')




Общая информация о таблице:

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           607 non-null    int64 
 1   experience_level    607 non-null    object
 2   employment_type     607 non-null    object
 3   job_title           607 non-null    object
 4   salary              607 non-null    int64 
 5   salary_currency     607 non-null    object
 6   salary_in_usd       607 non-null    int64 
 7   employee_residence  607 non-null    object
 8   remote_ratio        607 non-null    int64 
 9   company_location    607 non-null    object
 10  company_size        607 non-null    object
dtypes: int64(4), object(7)
memory usage: 56.9+ KB

Размерность таблицы: (607, 11)


Статистические характеристики признаков:



,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
count,607.000000,607,607,607,6.070000e+02,607,607.000000,607,607.00000,607,607
unique,NaN,4,4,50,NaN,17,NaN,57,NaN,50,3
top,NaN,SE,FT,Data Scientist,NaN,USD,NaN,US,NaN,US,M
freq,NaN,280,588,143,NaN,398,NaN,332,NaN,355,326
mean,2021.405272,NaN,NaN,NaN,3.240001e+05,NaN,112297.869852,NaN,70.92257,NaN,NaN
std,0.692133,NaN,NaN,NaN,1.544357e+06,NaN,70957.259411,NaN,40.70913,NaN,NaN
min,2020.000000,NaN,NaN,NaN,4.000000e+03,NaN,2859.000000,NaN,0.00000,NaN,NaN
25%,2021.000000,NaN,NaN,NaN,7.000000e+04,NaN,62726.000000,NaN,50.00000,NaN,NaN
50%,2022.000000,NaN,NaN,NaN,1.150000e+05,NaN,101570.000000,NaN,100.00000,NaN,NaN
75%,2022.000000,NaN,NaN,NaN,1.650000e+05,NaN,150000.000000,NaN,100.00000,NaN,NaN


### Дубликаты

In [609]:
dupes = df[df.duplicated()]
print(f'Количество дубликатов: {dupes.shape[0]}')

# Удаляем дубликаты:
df = df.drop_duplicates()
print(f'\nРазмерность таблицы после удаления дубликатов: {df.shape}')

Количество дубликатов: 42

Размерность таблицы после удаления дубликатов: (565, 11)


### Пропуски

In [610]:
print('\nПроверка на наличие пропусков в столбцах:\n')
display(
    df.isnull().mean()
)


Проверка на наличие пропусков в столбцах:



work_year             0.0
experience_level      0.0
employment_type       0.0
job_title             0.0
salary                0.0
salary_currency       0.0
salary_in_usd         0.0
employee_residence    0.0
remote_ratio          0.0
company_location      0.0
company_size          0.0
dtype: float64

### Неинформативные признаки

In [611]:
threshold = 0.95

low_info_cols = []

for col in df.columns:

    top_freq = df[col].value_counts(normalize=True).max()

    nunique_ratio = df[col].nunique() / df[col].count()

    if top_freq > threshold:
        low_info_cols.append(col)
        print(f'{col}: {round(top_freq * 100, 2)}% одинаковых значений')

    if nunique_ratio > threshold:
        low_info_cols.append(col)
        print(f'{col}: {round(nunique_ratio * 100, 2)}% уникальных значений')

df = df.drop(low_info_cols, axis = 1)

str_cols = ', '.join(low_info_cols)
print(f'\nПризнаки удалены: {str_cols}')
print(f'\nРазмерность таблицы: {df.shape}\n')



employment_type: 96.64% одинаковых значений

Признаки удалены: employment_type

Размерность таблицы: (565, 10)



### Классификация признаков

In [612]:
numerical_cols = ['salary', 'salary_in_usd']

categorical_cols = [
    'work_year',
    'experience_level',
    'salary_currency',
    'employee_residence',
    'remote_ratio',
    'company_location',
    'company_size',
    'job_title'
    ]

n_str = '\n'.join(numerical_cols)
c_str = '\n'.join(categorical_cols)

print(f'\nЧисловые признаки:\n{n_str}')
print(f'\nКатегориальные признаки:\n{c_str}')

for c in categorical_cols:
    df[c] = df[c].astype('category')


Числовые признаки:
salary
salary_in_usd

Категориальные признаки:
work_year
experience_level
salary_currency
employee_residence
remote_ratio
company_location
company_size
job_title


### Обработка выбросов

После сравнения результатов от удаления выбросов и замены значений медианными в рамках задания принято решение заменить выбросы в признаках заработной платы соответствующими медианными значениями.

In [613]:
# удалять выбросы или заменять медианным значением
class Method:
    remove = 1,
    median = 2

def outliers_iqr(df, target, method, left = 1.5, right = 1.5):

    x = df[target]
    q1, q3 = x.quantile(0.25), x.quantile(0.75),
    iqr = q3 - q1
    lower_bound = q1 - (iqr * left)
    upper_bound = q3 + (iqr * right)
    outliers = df[(x < lower_bound) | (x > upper_bound)]

    if outliers is None or len(outliers) == 0:
        return df

    print(f'\nЧисло выбросов в признаке \'{target}\' по методу Тьюки c границами {left}, {right}: {outliers.shape[0]}')

    threshold = outliers[target].min()

    if method == Method.median:
        median_ = df.loc[df[target] <= threshold, target].median()
        df.loc[df[target] >= threshold, target] = median_
        print(f'\nЗаменим выбросы в признаке \'{target}\' медианным значением: {median_}')
        return df

    if method == Method.remove:
        print(f'\nУдалим записи со значением \'{target}\' >= {threshold}')
        mask = df[target] >= threshold
        return df[~mask]



left, right = 1.5, 1.5
method = Method.median
df = outliers_iqr(df, 'salary', method, left, right)
print('\n')
df = outliers_iqr(df, 'salary_in_usd', method, left, right)

   


Число выбросов в признаке 'salary' по методу Тьюки c границами 1.5, 1.5: 44

Заменим выбросы в признаке 'salary' медианным значением: 105000.0



Число выбросов в признаке 'salary_in_usd' по методу Тьюки c границами 1.5, 1.5: 10

Заменим выбросы в признаке 'salary_in_usd' медианным значением: 100000.0


### Выводы по обработке данных

Исходная таблица изначально содержала 607 строк и 11 столбцов.

В таблице обнаружено и удалено 42 дублирующиеся записи.

Обнаружен и удалён 1 неинформативный признак (employment_type).

После удаления неинформативных признаков и дубликатов осталось 565 строк и 10 столбцов.

В целевых признаках salary и salary_in_usd обнаружено 44 и 10 выбросов соответственно.

В результирующей таблице насчитывается 2 числовых признака (salary, salary_in_usd) и 8 категориальных (work_year, experience_level, salary_currency,employee_residence, remote_ratio, company_location, company_size, job_title)

# 2. Разведывательный анализ данных

## 2.1 Визуальный анализ

In [614]:
# общий метод визуализации признаков (гистограмма, столбчатая диаграмма)
def grid_plot(df, columns, row_count, col_count, type, width = 1000, height = 1000, title = ''):

    c = 1
    r = 1

    plot = make_subplots(rows = row_count, cols = col_count)

    for i, t in enumerate(columns):

        if i != 0 and i % col_count == 0:
            r += 1
            c = 1

        if type == 'numerical':
            trace = go.Histogram(x = df[t], name = locale[t])
            plot.add_trace(trace, row = r, col = c)

        elif type == 'categorical':
            nunique = df[t].nunique()
            category = df[t].value_counts()
            index = [locale.get(i, i) for i in category.index] if t not in ['employee_residence', 'company_location'] else category.index
            name = f'{locale[t]} ({nunique})'
            trace = go.Bar(x = index, y = category.values, text = category.values, name = name)
            plot.add_trace(trace, row = r, col = c)
            plot.update_xaxes(tickmode = 'array', tickvals = index, ticktext = index, row=r, col=c)
        else:
            return

        if col_count > 1:
            c += 1

    plot.update_layout(height = height, title_text = title)

    render_plot.render(plot, file = f"images/{title}.png", width = width, height = height, type = current_render_type)


# общий метод визуализации признаков (box)
def grid_pair_box_plot(df, x_values, width = 1000, height = 1000):

    x_values.reverse()


    for t in x_values:

        title = ''

        plot = make_subplots(rows = 1, cols = 2)
        for c in range(1, 3):
            val = x_values.pop()
            if len(val) == 0:
                return
            title += f'{val}_'
            trace = go.Box(x = df['salary_in_usd'], y = df[val], text = df['salary_in_usd'], name = locale[val], orientation = 'h')
            plot.add_trace(trace, row = 1, col = c)
            category = df[val].value_counts()
            index = [locale.get(i, i) for i in category.index] if val not in ['employee_residence', 'company_location'] else category.index
            plot.update_yaxes(tickmode = 'array', tickvals = category.index, ticktext = index, row = 1, col = c)
            c+=1

            plot.update_layout(height = height)

        render_plot.render(plot, file = f"images/{title}_box.png", width = width, height = height, type = current_render_type)



### Гистограммы числовых признаков

In [615]:
# гистограммы числовых признаков
grid_plot(df, numerical_cols, 1, 2, 'numerical', 1000, 500, 'Гистограммы числовых признаков')

<div>
    <img src="images/Гистограммы числовых признаков.png">
</div>
<br/>

### Количество категориальных признаков

In [616]:
# количество категориальных признаков
grid_plot(df, categorical_cols, 4, 2, 'categorical', 2000, 1000, 'Количество категориальных признаков')

<div>
    <img src="images/Количество категориальных признаков.png">
</div>
<br/>

### Влияние признаков на зарплату (в USD) по всем должностям

In [617]:
grid_pair_box_plot(df, categorical_cols, width = 2000, height = 800)

<div>
    <img src="images/work_year_experience_level__box.png">
</div>
<br/>
<div>
    <img src="images/salary_currency_employee_residence__box.png">
</div>
<br/>
<div>
    <img src="images/remote_ratio_company_location__box.png">
</div>
<br/>

### Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist

In [618]:
mask = (df['job_title'] == 'Data Scientist')

data = df[mask].groupby('work_year').median('salary_in_usd')['salary_in_usd']

x = data.index
y = data.values

trace = go.Scatter(x = x, y = y, mode='lines+markers')
layout = go.Layout(title = 'Рост зарплаты у специалистов на должности Data Scientist')
plot = go.Figure(data=[trace], layout = layout)
plot.update_xaxes(tickmode = 'array', tickvals = x, ticktext = x)

render_plot.render(plot, file = f"images/ds_salary_grow.png", width = 1000, height = 500, type = current_render_type)


<div>
    <img src="images/ds_salary_grow.png">
</div>
<br/>

### Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году

In [619]:
mask = ((df['job_title'] == 'Data Scientist') | (df['job_title'] == 'Data Engineer') & (df['work_year'] == 2022))

data = df[mask]

colors = ['red', 'blue']

plot = go.Figure()

for i, title in enumerate(data['job_title'].unique()):
    df_plot = data[data['job_title'] == title]

    plot.add_trace(go.Box(
                        x = df_plot['salary_in_usd'],
                        y = df_plot['job_title'],
                        line=dict(color=colors[i]),
                        name= f'{title}',
                        orientation = 'h'
                        ),
                )

plot.update_layout(boxmode = 'group', title = 'Соотношение зарплаты Data Scientist и Data Engineer в 2022 году')

render_plot.render(plot, file = f"images/ds_de_salary_compare.png", width = 1000, height = 500, type = current_render_type)

<div>
    <img src="images/ds_de_salary_compare.png">
</div>
<br/>

### Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров

### Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании